In [8]:
!pip install mysql-connector-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.0/34.0 MB 35.5 MB/s eta 0:00:00


In [1]:
!pip install kafka-python==2.0.2
!pip show kafka-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 6.3 MB/s eta 0:00:00
Name: kafka-python
Version: 2.0.2
Summary: Pure Python client for Apache Kafka
Home-page: https://github.com/dpkp/kafka-python
Author: Dana Powers
Author-email: dana.powers@gmail.com
License: Apache License 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: 
Required-by: 


In [16]:
import mysql.connector
from google.colab import userdata

# Load credentials from secrets

MYSQL_HOST = userdata.get("MYSQL_HOST")
KAFKA_HOST = MYSQL_HOST
MYSQL_USER = userdata.get("MYSQL_USER")
MYSQL_PASS = userdata.get("MYSQL_PASS")

# Replace with your MySQL server details
conn = mysql.connector.connect(
    host=MYSQL_HOST,
    user=MYSQL_USER,
    password=MYSQL_PASS,
    database="iot"
)

cursor = conn.cursor()
print("Connected to MySQL!")


Connected to MySQL!


In [18]:
# Create database if it doesn't exist
cursor.execute("CREATE DATABASE IF NOT EXISTS iot")
cursor.execute("USE iot")

# Create table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS sensor_data (
        id INT AUTO_INCREMENT PRIMARY KEY,
        sensor_id VARCHAR(50),
        temperature FLOAT,
        pressure FLOAT,
        humidity FLOAT,
        battery_level FLOAT,
        location VARCHAR(100),
        sensor_type VARCHAR(50),
        device_status VARCHAR(50),
        timestamp DOUBLE
    )
""")

print("Database and table created successfully!")


Database and table created successfully!


In [13]:
cursor.execute("ALTER TABLE sensor_data MODIFY device_status VARCHAR(50);")

In [19]:
import json
from kafka import KafkaConsumer


KAFKA_BROKER = f"{KAFKA_HOST}:9093"

# Kafka Consumer
consumer = KafkaConsumer(
    "iot-sensors",
    bootstrap_servers= KAFKA_BROKER,
    auto_offset_reset='earliest',
    value_deserializer=lambda x: json.loads(x.decode('utf-8'))  # Convert JSON to dict
)

print("Listening for IoT Sensor Data...")

for message in consumer:
    data = message.value
    print(f"Received: {data}")

    # Insert Data into MySQL
    sql = """
    INSERT INTO sensor_data (sensor_id, temperature, pressure, humidity, battery_level, location, sensor_type, device_status, timestamp)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    values = (
        data["sensor_id"],
        data["temperature"],
        data["pressure"],
        data["humidity"],
        data["battery_level"],
        data["location"],
        data["sensor_type"],
        data["device_status"],
        data["timestamp"],
    )

    cursor.execute(sql, values)
    conn.commit()

print("Data successfully stored in MySQL!")

Streaming output truncated to the last 5000 lines.
Received: {'sensor_id': 'sensor-02', 'temperature': 20.18, 'pressure': 904.28, 'humidity': 59.01, 'battery_level': 97.42, 'location': 'Room B', 'sensor_type': 'Proximity', 'device_status': 'offline', 'timestamp': 1742071033.4979343}
Received: {'sensor_id': 'sensor-04', 'temperature': 20.98, 'pressure': 916.74, 'humidity': 33.19, 'battery_level': 27.48, 'location': 'Room A', 'sensor_type': 'Temperature', 'device_status': 'active', 'timestamp': 1742071033.4979815}
Received: {'sensor_id': 'sensor-03', 'temperature': 22.25, 'pressure': 922.94, 'humidity': 56.73, 'battery_level': 50.09, 'location': 'Factory Floor', 'sensor_type': 'Temperature', 'device_status': 'offline', 'timestamp': 1742071033.4980237}
Received: {'sensor_id': 'sensor-04', 'temperature': 27.7, 'pressure': 900.79, 'humidity': 52.07, 'battery_level': 37.07, 'location': 'Room A', 'sensor_type': 'Pressure', 'device_status': 'offline', 'timestamp': 1742071033.498066}
Received: 

InterfaceError: 2013: Lost connection to MySQL server during query

curl: (56) Recv failure: Connection reset by peer


In [15]:
from google.colab import userdata

# Save credentials (Run this once & manually enter details)
userdata.set("MYSQL_HOST", "34.123.174.89")  # Your MySQL VM IP
userdata.set("MYSQL_USER", "root")
userdata.set("MYSQL_PASS", "MakeAmericaGreatAgain@123")
userdata.set("MYSQL_DB", "iot")

AttributeError: module 'google.colab.userdata' has no attribute 'set'